In [1]:
# Import our custom modules
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import shutil

# Add the parent directory to the system path (if needed)
# Uncomment these lines if the modules are in a parent directory
# sys.path.insert(0, os.path.abspath(os.path.join(os.path.dirname('__file__'), '..')))

# Import our custom modules
from submesh.submesh import Mesh, RectangleMask, CircleMask, ShapefileMask, ArbitrMask
from submesh.io import read_fort14, read_fort63_nc, save_netcdf, read_output, output_to_xarray
from submesh.plotting import plot_subgrid_overlay, plot_full_mesh, plot_elevation

In [2]:
# Path to your fort.14 file
# mesh_path = "path/to/your/fort.14"
PATH = '../data/OCT2024/plug/'
mesh_path = os.path.join(PATH, 'fort.14')

# Option 1: Use the Mesh class (recommended)
mesh = Mesh.from_fort14(mesh_path)

In [3]:
# Path to another mesh file that defines the region of interest
# arbitrary_mesh_path = "path/to/arbitrary_region.14"
# rect_mesh_path = '/Users/crj85568/Projects/KingsBay/data/test.14'

# Create an arbitrary mesh submesh
try:
    rect_mesh = RectangleMask( -81.7,30.6, -81.3,  31.0)
    submesh_rect = mesh.subset(rect_mesh, folder=PATH)
    
    print(f"Arbitrary mesh submesh: {len(submesh_rect.nodes)} nodes, {len(submesh_rect.elements)} elements")
    
   
    submesh_rect.to_fort14(os.path.join(PATH, 'PTM_plug.14'))
except FileNotFoundError:
    print(f"Arbitrary mesh file not found: {arbitrary_mesh_path}")

Arbitrary mesh submesh: 207111 nodes, 413201 elements
✅ fort.14 written to ../data/OCT2024/plug/PTM_plug.14


In [4]:
# The node ID mapping should be saved during the submesh creation
# shutil.move('arbitr/node_id_map.csv', os.path.join(PATH, 'node_id_map.csv'))
mapping_file = os.path.join(PATH, "node_id_map.csv")

# If you need to manually load it:
mapping_df = pd.read_csv(mapping_file)
print("Node ID mapping (first 5 rows):")
print(mapping_df.head())

Node ID mapping (first 5 rows):
   mesh_node_id  subset_node_id
0         44728               1
1         44729               2
2         44730               3
3         45037               4
4         45038               5


In [5]:
### Merge mesh back together
from submesh import merge
submesh = Mesh.from_fort14('../data/OCT2024/oct/PTM_region_rect.14')
merge.merge_domain(submesh, mesh)


MERGE DOMAIN - Progress Tracking
Mesh size: 1,689,559 nodes, 3,367,090 elements

[Step 1/10] Finding global grid boundary (netable=all 1s)...
              mkeline: Processing 3,367,090/3,367,090 elements
              mkeline: Created edges (0.1s)
              mkeline: Found 2,514,559 boundary edges (1.5s total)
            Found 2,514,559 global boundary edges (1.5s)

[Step 2/10] Creating netable from node_id_map.csv...
            413,213 elements in subgrid region (34.4s)

[Step 3/10] Finding subgrid boundary edges...
              mkeline: Processing 413,213/3,367,090 elements
              mkeline: Created edges (0.0s)
              mkeline: Found 312,956 boundary edges (0.7s total)
            Found 312,956 subgrid boundary edges (0.7s)

[Step 4/10] Comparing global vs subgrid boundaries...
            Marked boundary differences (0.6s)

[Step 5/10] Processing connected boundaries...
            Processed 0 connected boundary segments (0.1s)

[Step 6/10] Creating final boundar

KeyError: 'new_index'

In [12]:
# Paths to your output files
fort63_nc_path = os.path.join(PATH, 'fort.63.nc') #data/plug_cutoff/fort.63.nc' # "path/to/fort.63.nc"  # If you have NetCDF output
from submesh.submesh import subset_output

# Process NetCDF fort.63 file
try: 
    output_file = os.path.join(PATH, "PTM_63.nc")
    subset_output(mapping_file, fort63_nc_path, output_file)
    print(f"Processed NetCDF fort.63 to {output_file}")
except Exception as e:
    print(f"File not found: {fort63_nc_path}, {e}")

📊 Processing NetCDF file: ../data/OCT2024/plug/fort.63.nc
ds_subset.node values (first 5): [0 1 2 3 4]
✅ Subset data saved to ../data/OCT2024/plug/PTM_63.nc
Processed NetCDF fort.63 to ../data/OCT2024/plug/PTM_63.nc


In [13]:
# Paths to your output files

fort64_nc_path = os.path.join(PATH, 'fort.64.nc') # "path/to/fort.63.nc"  # If you have NetCDF output
from submesh.submesh import subset_output

# Process NetCDF fort.63 file
try: 
    output_file = os.path.join(PATH, "PTM_64.nc")
    subset_output(mapping_file, fort64_nc_path, output_file)
    print(f"Processed NetCDF fort.64 to {output_file}")
except FileNotFoundError:
    print(f"File not found: {fort64_nc_path}")

📊 Processing NetCDF file: ../data/OCT2024/plug/fort.64.nc
ds_subset.node values (first 5): [0 1 2 3 4]
✅ Subset data saved to ../data/OCT2024/plug/PTM_64.nc
Processed NetCDF fort.64 to ../data/OCT2024/plug/PTM_64.nc


In [19]:
ds = xr.open_dataset(os.path.join('../data/OCT2024/oct', 'fort.63.nc'))
ds

<xarray.Dataset> Size: 18GB
Dimensions:      (time: 1296, node: 1689559, nfaces: 3367090, nvertex: 3,
                  nope: 1, neta: 292, nbou: 111, nvel: 12066)
Coordinates:
  * time         (time) datetime64[ns] 10kB 2024-10-08T01:00:00 ... 2024-12-01
    x            (node) float64 14MB ...
    y            (node) float64 14MB ...
Dimensions without coordinates: node, nfaces, nvertex, nope, neta, nbou, nvel
Data variables:
    element      (nfaces, nvertex) int32 40MB ...
    adcirc_mesh  int32 4B ...
    nvdll        (nope) int32 4B ...
    ibtypee      (nope) int32 4B ...
    nbdv         (neta) int32 1kB ...
    nvell        (nbou) int32 444B ...
    ibtype       (nbou) int32 444B ...
    nbvv         (nvel) int32 48kB ...
    depth        (node) float64 14MB ...
    zeta         (time, node) float64 18GB ...
Attributes: (12/50)
    _FillValue:         -99999.0
    model:              ADCIRC
    version:            v56.0.4
    git_hash:           5e5905d7e4b330b5c4c263d4253136c3199155f3
    grid_type:          Triangular
    description:        SAB          ! 32 CHARACTER ALPHANUMERIC RUN DESCRIPTION
    ...                 ...
    sfea0:              35.0
    cf:                 0.0025
    eslm:               10.0
    cori:               0.0
    ntif:               7
    nbfr:               7